In [ ]:
#!/usr/bin/env python
# coding: utf-8

# Import necessary libraries
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import yfinance as yf
from datetime import date, timedelta
import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np

In [ ]:
# Class to store news information
class NewsInformation:
    """Class to store news information."""
    title = description = ""

In [ ]:
# Class for HTML parsing
class NewsParser:
    """Class for HTML parsing and extracting news information."""
    def get_news(self, url):
        """Get news information from a given URL."""
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        news_html_list = soup.find_all(class_="newsreleaseconsolidatelink display-outline")

        news_list = []
        count = 1
        for rows in news_html_list:
            news = NewsInformation()

            # Get description from node
            description = self.get_description(rows)
            if description is not None:
                news.description = description.get_text()

            # Get title from node
            title_with_time = self.get_title_from_root(rows)
            if title_with_time is not None:
                news.title = title_with_time.get_text()
                if title_with_time.small is not None:
                    time = title_with_time.small.get_text()
                    # Remove time from title strings
                    if news.title.strip() != "" and time.strip() != "":
                        news.title = news.title.replace(time, '')

            news_list.append(news)
            count += 1
        return news_list

    def get_title_from_root(self, news_root):
        """Get the title of the news from the HTML node."""
        title = None
        if news_root.find(class_="col-sm-8 col-lg-9 pull-left card") is not None:
            title = news_root.find(class_="col-sm-8 col-lg-9 pull-left card").h3
        elif news_root.find(class_="no-top-margin remove-outline") is not None:
            title = news_root.find(class_="no-top-margin remove-outline")
        return title

    def get_description(self, news_root):
        """Get the description of the news from the HTML node."""
        description = None
        result = news_root.find_all(lambda tag: tag.name == 'p' and tag.get('class') == ['remove-outline'])
        if len(result) > 0:
            description = result[0]
        return description

In [ ]:
# Class for general utility functions
class Utils:
    """Class for general utility functions."""
    def get_stock_symbols(self, news_list):
        """Get stock symbols from a list of news information."""
        stock_symbols = []
        for news in news_list:
            # Fetched symbol from title
            symbol = self.find_symbol(news.title)
            if symbol is not None and symbol not in stock_symbols:
                stock_symbols.append(symbol)

            # Fetched symbol from description
            symbol = self.find_symbol(news.description)
            if symbol is not None and symbol not in stock_symbols:
                stock_symbols.append(symbol)
        return stock_symbols

    def find_symbol(self, text):
        """Find stock symbols in a given text."""
        regex_output = re.findall(r'\([A-Z]*:\s*[A-Z]*\)', text)
        if len(regex_output) > 0:
            exchange_stock_names = regex_output[0].replace("(", "").replace(")", "").split(":")
            if len(exchange_stock_names) > 1:
                return exchange_stock_names[1].strip()

In [ ]:
# Class to retrieve stock information from Yahoo Finance
class StockInformation:
    """Class to retrieve stock information from Yahoo Finance."""
    def __init__(self, stock_symbols):
        self.stock_symbols = stock_symbols

    def get_stock_info(self):
        """Get stock information for selected stocks from Yahoo Finance."""
        selected_stocks = self.select_stocks()
        start_date = date.today() + timedelta(days=-180)  # 6 months
        end_date = date.today()
        stocks_data = pd.DataFrame()

        for symbol in selected_stocks:
            try:
                print("Downloading: " + symbol)
                stock_data = yf.download(symbol, start=start_date, end=end_date)

                if len(stock_data) == 0:
                    continue

                stock_data['Name'] = symbol
                stock_data['20_EMA'] = stock_data['Close'].rolling(window=20, min_periods=1).mean()
                stock_data['50_EMA'] = stock_data['Close'].rolling(window=50, min_periods=1).mean()
                stock_data['Signal'] = np.where(stock_data['20_EMA'] > stock_data['50_EMA'], 1.0, 0.0)
                stock_data['Position'] = stock_data['Signal'].diff()

                stocks_data = pd.concat([stocks_data, stock_data])

            except Exception as e:
                print(f"Error downloading {symbol}: {e}")

        return stocks_data

    def select_stocks(self):
        """Select the first 4 stocks."""
        return self.stock_symbols[:4] if len(self.stock_symbols) >= 4 else self.stock_symbols

In [ ]:
# Class for graphical representation of stock data
class GraphUtils:
    """Class for graphical representation of stock data."""
    def draw_graphs(self, selected_stocks, stocks_data):
        """Draw candlestick graphs for selected stocks."""
        for symbol in selected_stocks:
            mpf.plot(stocks_data.loc[stocks_data['Name'] == symbol],
                     type='candle',
                     mav=(20, 50), volume=True,
                     title=symbol,
                     style='yahoo')

    def draw_signal_graphs(self, selected_stocks, stocks_data):
        """Draw graphs with buy/sell signals for selected stocks."""
        for symbol in selected_stocks:
            stock_data = stocks_data.loc[stocks_data['Name'] == symbol]

            plt.figure(figsize=(10, 5))
            stock_data['Close'].plot(color='k', lw=1, label='Close')
            stock_data['20_EMA'].plot(color='b', lw=1, label='20-day EMA')
            stock_data['50_EMA'].plot(color='g', lw=1, label='50-day EMA')
            plt.plot(stock_data[stock_data['Position'] == 1].index,
                     stock_data['20_EMA'][stock_data['Position'] == 1],
                     '^', markersize=15, color='g', label='Buy')
            plt.plot(stock_data[stock_data['Position'] == -1].index,
                     stock_data['20_EMA'][stock_data['Position'] == -1],
                     'v', markersize=15, color='r', label='Sell')
            plt.ylabel('Price', fontsize=15)
            plt.xlabel('Date', fontsize=15)
            plt.title(symbol + ' - EMA Crossover', fontsize=20)
            plt.legend()
            plt.grid()
            plt.show()

    def analyze_stocks(self, stocks_data):
        """Analyze stock data and provide recommendations."""
        for symbol in stocks_data['Name'].unique():
            stock_subset = stocks_data[stocks_data['Name'] == symbol]
            initial_price = stock_subset['Close'].iloc[0]
            final_price = stock_subset['Close'].iloc[-1]
            percentage_change = ((final_price - initial_price) / initial_price) * 100

            print(f"\nStock: {symbol}")
            print(f"Initial Price: ${initial_price:.2f}")
            print(f"Final Price: ${final_price:.2f}")
            print(f"Percentage Change: {percentage_change:.2f}%")

            # Simple strategy: Buy if percentage change is positive, Sell if negative
            if percentage_change > 0:
                print("Recommendation: Buy")
            else:
                print("Recommendation: Sell")

In [ ]:
# Main execution
if __name__ == "__main__":
    # Step 1: Get news information
    news_url = "https://www.prnewswire.com/news-releases/news-releases-list/?page=1&pagesize=100"
    news_parser = NewsParser()
    news_list = news_parser.get_news(news_url)

    # Step 2: Get stock symbols from news information
    utils = Utils()
    stock_symbols = utils.get_stock_symbols(news_list)

    print("\nStock Symbols in News List", len(stock_symbols))
    print("\nStock Symbols", stock_symbols)

    # Step 3: Get stock information from Yahoo Finance
    stock_info = StockInformation(stock_symbols)
    selected_stocks = stock_info.select_stocks()
    print("Selected Stocks ", selected_stocks)

    stocks_data = stock_info.get_stock_info()
    print(stocks_data.head())

    # Step 4: Visualize stock data
    graph_utils = GraphUtils()
    graph_utils.draw_graphs(selected_stocks, stocks_data)
    graph_utils.draw_signal_graphs(selected_stocks, stocks_data)

    # Analyze stocks and provide recommendations
    graph_utils.analyze_stocks(stocks_data)